In [1]:
import os
import pickle
import librosa
import numpy as np
import scipy.signal
from tqdm import tqdm

def extract_features(file_path, frame_size=0.05, frame_stride=0.02, n_fft=512):
    try:
        y, sr = librosa.load(file_path, sr=None)

        # Pre-emphasis filter
        pre_emphasis = 0.97
        emphasized_signal = np.append(y[0], y[1:] - pre_emphasis * y[:-1])

        # STFT parameters
        hop_length = int(frame_stride * sr)
        frequencies, times, Zxx = scipy.signal.stft(emphasized_signal, fs=sr, nperseg=n_fft, noverlap=n_fft - hop_length, window="hann")

        # Magnitude Spectrogram (STFT)
        magnitude_spectrogram = np.abs(Zxx)
        log_magnitude_spectrogram = np.log(magnitude_spectrogram + 1e-10)

        # Chroma Features
        chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=hop_length, n_fft=n_fft)

        # Chroma Delta (đạo hàm bậc 1)
        chroma_delta = librosa.feature.delta(chroma)

        return log_magnitude_spectrogram, chroma, chroma_delta, None

    except Exception as e:
        return None, None, None, str(e)

def extract_features_from_folder(root_folder):
    all_data = []
    errors = []

    for dirpath, _, filenames in os.walk(root_folder):
        for file in tqdm(filenames, desc=f"Processing {dirpath}", leave=False):
            if file.endswith(('.wav', '.mp3')):
                file_path = os.path.join(dirpath, file)
                relative_path = os.path.relpath(file_path, root_folder)
                parts = relative_path.split(os.sep)

                category = parts[0] if len(parts) > 0 else "Unknown"
                subcategory = parts[1] if len(parts) > 1 else "Unknown"

                stft, chroma, chroma_delta, error = extract_features(file_path)
                if stft is not None and chroma is not None and chroma_delta is not None:
                    feature_row = {
                        'file_name': file,
                        'category': category,
                        'subcategory': subcategory,
                        'stft': stft,
                        'chroma': chroma,
                        'chroma_delta': chroma_delta
                    }
                    all_data.append(feature_row)
                else:
                    errors.append((file_path, error))

    return all_data, errors

def save_to_pickle(data, output_file):
    with open(output_file, 'wb') as f:
        pickle.dump(data, f)

if __name__ == "__main__":
    root_folder = r"C:\Users\Admin\Documents\Python Project\bee-swarming\SB1\buzz 1"
    output_folder = r"C:\Users\Admin\Documents\Python Project\bee-swarming\STFT Chroma\STFT_Chroma_features_buzz1"
    
    os.makedirs(output_folder, exist_ok=True)

    for subset in ["Test", "Train", "Val"]:
        subset_path = os.path.join(root_folder, subset)
        print(f"Processing {subset_path}...")

        features_data, errors = extract_features_from_folder(subset_path)

        output_file = os.path.join(output_folder, f"{subset}_features.pkl")
        save_to_pickle(features_data, output_file)
        print(f"Saved features for {subset} to {output_file}")

        if errors:
            error_log_file = os.path.join(output_folder, f"{subset}_errors.log")
            with open(error_log_file, "w", encoding="utf-8") as f:
                for file_path, error in errors:
                    f.write(f"{file_path}: {error}\n")
            print(f"Logged errors for {subset} in {error_log_file}")


Processing C:\Users\Admin\Documents\Python Project\bee-swarming\SB1\buzz 1\Test...


Processing C:\Users\Admin\Documents\Python Project\bee-swarming\SB1\buzz 1\Test\None_swarming\1626712776797_Dan 02, 07cau, co chua, troi nang_190721:   8%|▊         | 61/800 [00:02<00:12, 58.83it/s]c:\Users\Admin\anaconda3\envs\tf\lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


Saved features for Test to C:\Users\Admin\Documents\Python Project\bee-swarming\STFT Chroma\STFT_Chroma_features_buzz1\Test_features.pkl
Processing C:\Users\Admin\Documents\Python Project\bee-swarming\SB1\buzz 1\Train...


Saved features for Train to C:\Users\Admin\Documents\Python Project\bee-swarming\STFT Chroma\STFT_Chroma_features_buzz1\Train_features.pkl
Processing C:\Users\Admin\Documents\Python Project\bee-swarming\SB1\buzz 1\Val...


Saved features for Val to C:\Users\Admin\Documents\Python Project\bee-swarming\STFT Chroma\STFT_Chroma_features_buzz1\Val_features.pkl


In [2]:
import os
import pickle
import librosa
import numpy as np
import scipy.signal
from tqdm import tqdm

def extract_features(file_path, frame_size=0.05, frame_stride=0.02, n_fft=512):
    try:
        y, sr = librosa.load(file_path, sr=None)

        # Pre-emphasis filter
        pre_emphasis = 0.97
        emphasized_signal = np.append(y[0], y[1:] - pre_emphasis * y[:-1])

        # STFT parameters
        hop_length = int(frame_stride * sr)
        frequencies, times, Zxx = scipy.signal.stft(emphasized_signal, fs=sr, nperseg=n_fft, noverlap=n_fft - hop_length, window="hann")

        # Magnitude Spectrogram (STFT)
        magnitude_spectrogram = np.abs(Zxx)
        log_magnitude_spectrogram = np.log(magnitude_spectrogram + 1e-10)

        # Chroma Features
        chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=hop_length, n_fft=n_fft)

        # Delta (1st derivative) of Chroma
        chroma_delta = librosa.feature.delta(chroma)

        return log_magnitude_spectrogram, chroma, chroma_delta, None

    except Exception as e:
        return None, None, None, str(e)

def extract_features_from_folder(root_folder):
    all_data = []
    errors = []

    for dirpath, _, filenames in os.walk(root_folder):
        for file in tqdm(filenames, desc=f"Processing {dirpath}", leave=False):
            if file.endswith(('.wav', '.mp3')):
                file_path = os.path.join(dirpath, file)
                relative_path = os.path.relpath(file_path, root_folder)
                parts = relative_path.split(os.sep)

                category = parts[0] if len(parts) > 0 else "Unknown"
                subcategory = parts[1] if len(parts) > 1 else "Unknown"

                stft, chroma, chroma_delta, error = extract_features(file_path)
                if stft is not None and chroma is not None and chroma_delta is not None:
                    feature_row = {
                        'file_name': file,
                        'category': category,
                        'subcategory': subcategory,
                        'stft': stft,
                        'chroma': chroma,
                        'chroma_delta': chroma_delta
                    }
                    all_data.append(feature_row)
                else:
                    errors.append((file_path, error))

    return all_data, errors

def save_to_pickle(data, output_file):
    with open(output_file, 'wb') as f:
        pickle.dump(data, f)

if __name__ == "__main__":
    root_folder = r"C:\Users\Admin\Documents\Python Project\bee-swarming\SB2\buzz 2"
    output_folder = r"C:\Users\Admin\Documents\Python Project\bee-swarming\STFT Chroma\STFT_Chroma_features_buzz2"
    
    os.makedirs(output_folder, exist_ok=True)

    for subset in ["Test", "Train", "Val"]:
        subset_path = os.path.join(root_folder, subset)
        print(f"Processing {subset_path}...")

        features_data, errors = extract_features_from_folder(subset_path)

        output_file = os.path.join(output_folder, f"{subset}_features.pkl")
        save_to_pickle(features_data, output_file)
        print(f"Saved features for {subset} to {output_file}")

        if errors:
            error_log_file = os.path.join(output_folder, f"{subset}_errors.log")
            with open(error_log_file, "w", encoding="utf-8") as f:
                for file_path, error in errors:
                    f.write(f"{file_path}: {error}\n")
            print(f"Logged errors for {subset} in {error_log_file}")


Processing C:\Users\Admin\Documents\Python Project\bee-swarming\SB2\buzz 2\Test...


Saved features for Test to C:\Users\Admin\Documents\Python Project\bee-swarming\STFT Chroma\STFT_Chroma_features_buzz2\Test_features.pkl
Processing C:\Users\Admin\Documents\Python Project\bee-swarming\SB2\buzz 2\Train...


Saved features for Train to C:\Users\Admin\Documents\Python Project\bee-swarming\STFT Chroma\STFT_Chroma_features_buzz2\Train_features.pkl
Processing C:\Users\Admin\Documents\Python Project\bee-swarming\SB2\buzz 2\Val...


Saved features for Val to C:\Users\Admin\Documents\Python Project\bee-swarming\STFT Chroma\STFT_Chroma_features_buzz2\Val_features.pkl


In [3]:
import soundfile as sf
import os

# Đường dẫn thư mục chứa âm thanh
root_folder = r"C:\Users\Admin\Documents\Python Project\bee-swarming\SB1\buzz 1\Test\None_swarming\1626712776797_Dan 02, 07cau, co chua, troi nang_190721"

# Lấy danh sách file âm thanh
audio_files = [f for f in os.listdir(root_folder) if f.endswith(('.wav', '.flac'))]

# Chọn file đầu tiên trong danh sách (hoặc chọn theo cách bạn muốn)
if audio_files:
    audio_path = os.path.join(root_folder, audio_files[0])
    
    # Đọc file âm thanh
    with sf.SoundFile(audio_path) as f:
        bit_depth = f.subtype_info
        print(f"File: {audio_files[0]}")
        print(f"Bit Depth: {bit_depth}")
else:
    print("Không tìm thấy file âm thanh trong thư mục.")



File: 1626712776797_Dan 02, 07cau, co chua, troi nang_190721_0.wav
Bit Depth: Signed 16 bit PCM
